In [1]:
import os
from PIL import Image, ImageOps
import numpy as np
import random
from scipy import ndimage
import cv2

def load_images(directory, target_size):
    images = []
    labels = []
    class_names = os.listdir(directory) # class names as per the folder name
    # creation of dictionary for enumeration
    class_to_index = {class_name: i for i, class_name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(directory, class_name + "/train")
        class_label = class_to_index[class_name]
        for filename in os.listdir(class_dir):
            image_path = os.path.join(class_dir, filename)
            image = Image.open(image_path).convert("RGB")
            
            padded_image = ImageOps.pad(image, target_size, method=Image.Resampling.LANCZOS)

            images.append(np.array(padded_image))
            labels.append(np.array(class_label))

    return images, labels

def augment_images(images, labels):
    augmented_images = []
    augmented_labels = []

    for i, image in enumerate(images):
        
        augmented_images.append(image)
        augmented_labels.append(labels[i])

        # Random horizontal flip
        if np.random.rand() < 0.3:
            flipped_image = np.fliplr(image)
            augmented_images.append(flipped_image)
            augmented_labels.append(labels[i])

        # Random rotation
        if np.random.rand() < 0.3:
            angle = np.random.randint(-15, 15)  # Rotate between -15 and +15 degrees
            rotated_image = ndimage.rotate(image, angle, reshape=False, mode='nearest')
            augmented_images.append(rotated_image)
            augmented_labels.append(labels[i])

        # add gaussian noise
        if np.random.rand() < 0.3:
            noise = np.random.normal(0, 0.1, image.shape)
            noisy_image = image + noise
            noisy_image = np.clip(noisy_image, 0, 1)
            augmented_images.append(noisy_image)
            augmented_labels.append(labels[i])

    return augmented_images, augmented_labels

data_dir = '/data/animal_data'
target_size = (195, 195)

images, labels = load_images(data_dir, target_size)
images, labels = augment_images(images, labels)

In [3]:
len(images)

12270

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

images = np.array(images)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# normalize
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# labels to one-hot encoding
num_classes = len(np.unique(labels))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

batch_size = 16

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

def cnn_model(num_classes):
    model = models.Sequential()

    # Convolutional layers
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())

    # Fully connected layers
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))

    # Output layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [4]:
model = cnn_model(num_classes) #created a CNN model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 178, 178, 16)      448       
_________________________________________________________________
batch_normalization (BatchNo (None, 178, 178, 16)      64        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 32)        4640      
_________________________________________________________________
batch_normalization_1 (Batch (None, 87, 87, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 64)        1

In [5]:
model.compile(optimizer=Adam(learning_rate=0.005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=batch_size, validation_split=0.2, verbose=2)

Train on 6097 samples, validate on 1525 samples
Epoch 1/10
6097/6097 - 188s - loss: 3.9839 - accuracy: 0.2864 - val_loss: 1.3843 - val_accuracy: 0.2728
Epoch 2/10
6097/6097 - 180s - loss: 1.4019 - accuracy: 0.2629 - val_loss: 1.3855 - val_accuracy: 0.2557
Epoch 3/10
6097/6097 - 200s - loss: 1.4055 - accuracy: 0.2434 - val_loss: 1.3837 - val_accuracy: 0.2656
Epoch 4/10
6097/6097 - 180s - loss: 1.4089 - accuracy: 0.2570 - val_loss: 1.3837 - val_accuracy: 0.2557
Epoch 5/10


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print("Test accuracy:", test_acc)